<a href="https://colab.research.google.com/github/Success-Vera/Gaussian-Mixture-Models/blob/main/master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reading Libraries

## Run these cells without changing anything

In [1]:
user = "Success-Vera"
repo = "Gaussian-Mixture-Models"
# src_dir = "src"
pyfile = "frequency.py"
prepfile='preprocessing.py'
svd='SVD.py'

url = f"https://raw.githubusercontent.com/{user}/{repo}/main/{pyfile}"
!wget --no-cache --backups=1 {url}

url2 = f"https://raw.githubusercontent.com/{user}/{repo}/main/{prepfile}"
!wget --no-cache --backups=1 {url2}

url3 = f"https://raw.githubusercontent.com/{user}/{repo}/main/{svd}"
!wget --no-cache --backups=1 {url3}

--2022-04-22 04:39:35--  https://raw.githubusercontent.com/Success-Vera/Gaussian-Mixture-Models/main/frequency.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3380 (3.3K) [text/plain]
Saving to: ‘frequency.py’

frequency.py        100%[===================>]   3.30K  --.-KB/s    in 0s      

2022-04-22 04:39:35 (33.5 MB/s) - ‘frequency.py’ saved [3380/3380]

--2022-04-22 04:39:35--  https://raw.githubusercontent.com/Success-Vera/Gaussian-Mixture-Models/main/preprocessing.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13630 (

In [44]:
import subprocess

result = subprocess.run(["wget", "--no-cache", "--backups=1", url], stderr=subprocess.PIPE, stdout=subprocess.PIPE)
result2 = subprocess.run(["wget", "--no-cache", "--backups=1", url2], stderr=subprocess.PIPE, stdout=subprocess.PIPE)
result3 = subprocess.run(["wget", "--no-cache", "--backups=1", url2], stderr=subprocess.PIPE, stdout=subprocess.PIPE)

# print(result2.stderr.decode("utf-8"))

## Importing Libraries

In [3]:
from preprocessing import forward
from SVD import *
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [4]:
#put in the path clean csv here. The csv is in the shared drive
data=pd.read_csv('/content/clean.csv')
data.head()

,review,sentiment,clean_text
0,One of the other reviewers has mentioned that ...,positive,One review ment watc 1 Oz episod ll hook Thei ...
1,A wonderful little production. <br /><br />The...,positive,A wond littl product br br The fil techniqu un...
2,I thought this was a wonderful way to spend ti...,positive,I thought wond wai spend time hot summ weekend...
3,Basically there's a family where a little boy ...,negative,Bas famili littl boi Jake think zombi closet p...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,Pett Mattei Love Time Monei visu stun film wat...


In [19]:
#you can uncomment this if you want to clean with the preprocessing module
# data['clean_text']=data['review'].apply(lambda X: forward(X))
# data.head()

In [5]:
data.drop('review',inplace=True,axis=1)

In [6]:
data['sentiment'].replace({'negative':0,"positive":1},inplace=True)

In [9]:
data=data[['clean_text','sentiment']]

In [8]:
data_set = data[['clean_text','sentiment']].copy()
index = data_set.index
data_set['random_number'] = np.random.randn(len(index))
train = data_set[data_set['random_number'] <= 0.8]
test = data_set[data_set['random_number'] > 0.8]


In [10]:
train.shape,test.shape

((39395, 3), (10605, 3))

In [11]:
train.head()

,clean_text,sentiment,random_number
0,One review ment watc 1 Oz episod ll hook Thei ...,1,-1.126293
1,A wond littl product br br The fil techniqu un...,1,0.577784
2,I thought wond wai spend time hot summ weekend...,1,-1.805516
4,Pett Mattei Love Time Monei visu stun film wat...,1,-0.855255
6,I sure would like see resurrect date Seahunt s...,1,0.262976


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(train)
tf_x_test = vectorizer.transform(test)

In [2]:
new_data=SVD.svd_decompose(tf_x_train,6)


In [28]:
new_data_test=svd_decompose(tf_x_test,6)

In [29]:
new_data_test.shape

(12500, 6)

In [40]:
import numpy as np
from scipy.stats import multivariate_normal

class GMM:
    def __init__(self, k, max_iter=5):
        self.k = k
        self.max_iter = int(max_iter)

    def initialize(self, X):
        self.shape = X.shape
        self.n, self.m = self.shape

        self.phi = np.full(shape=self.k, fill_value=1/self.k)
        self.weights = np.full( shape=self.shape, fill_value=1/self.k)
        
        random_row = np.random.randint(low=0, high=self.n, size=self.k)
        self.mu = [  X[row_index,:] for row_index in random_row ]
        self.sigma = [ np.cov(X.T) for _ in range(self.k) ]

    def e_step(self, X):
        # E-Step: update weights and phi holding mu and sigma constant
        self.weights = self.predict_proba(X)
        self.phi = self.weights.mean(axis=0)
    
    def m_step(self, X):
        # M-Step: update mu and sigma holding phi and weights constant
        for i in range(self.k):
            weight = self.weights[:, [i]]
            total_weight = weight.sum()
            self.mu[i] = (X * weight).sum(axis=0) / total_weight
            self.sigma[i] = np.cov(X.T, 
                aweights=(weight/total_weight).flatten(), 
                bias=True)

    def fit(self, X):
        self.initialize(X)
        
        for iteration in range(self.max_iter):
            self.e_step(X)
            self.m_step(X)
            
    def predict_proba(self, X):
        likelihood = np.zeros( (self.n, self.k) )
        for i in range(self.k):
            distribution = multivariate_normal(
                mean=self.mu[i], 
                cov=self.sigma[i])
            likelihood[:,i] = distribution.pdf(X)
        
        numerator = likelihood * self.phi
        denominator = numerator.sum(axis=1)[:, np.newaxis]
        weights = numerator / denominator
        return weights
    
    def predict(self, X):
        weights = self.predict_proba(X)
        return np.argmax(weights, axis=1)

In [41]:

gm=GMM(2)
gm.fit(new_data)

In [43]:
gm.predict(new_data_test)